In [25]:

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
import pickle

In [26]:
data = pd.read_csv('ДЗ_2_Прогнозирование_оценки_за_экз_по_ТОЭ.csv')
data = data.fillna(0) #заполним нулями пропуски
data.head()

,Отметка времени,Итоговая оценка за экзамен по ТОЭ,"С какого раза вы сдали экзамен по ТОЭ (если улучшали оценку, так же указать с какого раза получили желаемую оценку)","Регулярность посещения занятий. Оцените процент занятий, которые посетили в последнем семетре по ТОЭ (лекции+практика)?",Оценка по физике в школе,Оценка по физике в последнем семестре?,Оценка за ИДЗ по ТОЭ за семестр,Оценка по лабораторным ТОЭ,Вели конспект по занятиям? (лекции и/или практики),Выходили к доске на практике?,...,В течении предыдущего семестра были трудоустроены и/или проходили стажировку?,"Если работали/проходили стажировку, то какая ставка (загрузка):",Где проживаете?,"Есть кот дома? (в месте, где сейчас живете)",Есть родной брат/сестра?,Гражданство,Сколько в среднем играли в прошлом семестре? Укажите число в часах,Была ли стипендия в 3 семестре?,Оцените свое ментальное здоровье в прошлом семестре,"Оцените, насколько были истощены (уставши) к началу сессии"
0,2024/09/30 11:51:10 PM GMT+3,5.0,1.0,75,5,5.0,5.0,5.0,да,да,...,нет,0.00,общежитие 8,нет,да,KAZ,10,да,9,1
1,2024/09/30 11:53:32 PM GMT+3,3.0,1.0,75,4,5.0,4.0,5.0,да,нет,...,нет,0.00,дома,нет,да,RUS,20,нет,7,6
2,2024/09/30 11:53:33 PM GMT+3,3.0,1.0,75,5,4.0,5.0,5.0,да,нет,...,да,0.25,дома,да,да,RUS,168,да,7,3
3,2024/09/30 11:57:50 PM GMT+3,3.0,1.0,75,5,4.0,4.0,4.0,нет,нет,...,да,0.50,общежитие 8,нет,да,KAZ,18,нет,7,2
4,2024/09/30 11:59:56 PM GMT+3,4.0,1.0,100,4,3.0,4.0,5.0,нет,нет,...,нет,0.00,дома,нет,да,RUS,20,нет,10,5


In [27]:
signs = ['Регулярность посещения занятий. Оцените процент занятий, которые посетили в последнем семетре по ТОЭ (лекции+практика)?',
            'Оценка по физике в школе',
            'Оценка по физике в последнем семестре?', 
            'Оценка за ИДЗ по ТОЭ за семестр ',
            'Оценка по лабораторным ТОЭ',
            'Вели конспект по занятиям? (лекции и/или практики)', 
            'Выходили к доске на практике?',
            'В течении предыдущего семестра были трудоустроены и/или проходили стажировку?',
            'Если работали/проходили стажировку, то какая ставка (загрузка):', 
            'Есть кот дома? (в месте, где сейчас живете)',
            'Есть родной брат/сестра?',
            'Была ли стипендия в 3 семестре?',
            'Оцените свое ментальное здоровье в прошлом семестре',
            'Оцените, насколько были истощены (уставши) к началу сессии'
           ] #выберем признаки, которые могли повлиять на сдачу экзамена

In [28]:
# Преобразование категориальных признаков в числовые
def extract_hours(text):
    # Используем регулярное выражение для поиска числа в тексте
    match = re.search(r'(\d+)', str(text))
    if match:
        return int(match.group(1))
    return None

data['Вели конспект по занятиям? (лекции и/или практики)'] = data['Вели конспект по занятиям? (лекции и/или практики)'].map({'да': 1, 'нет': 0})
data['Выходили к доске на практике?'] = data['Выходили к доске на практике?'].map({'да': 1, 'нет': 0})
data['В течении предыдущего семестра были трудоустроены и/или проходили стажировку?'] = data['В течении предыдущего семестра были трудоустроены и/или проходили стажировку?'].map({'да': 1, 'нет': 0, '...': 0.5}) # Заменяем "..." на среднее значение
data['Есть кот дома? (в месте, где сейчас живете)'] = data['Есть кот дома? (в месте, где сейчас живете)'].map({'да': 1, 'нет': 0})
data['Есть родной брат/сестра?'] = data['Есть родной брат/сестра?'].map({'да': 1, 'нет': 0})
data['Была ли стипендия в 3 семестре?'] = data['Была ли стипендия в 3 семестре?'].map({'да': 1, 'нет': 0})
data['Сколько в среднем играли в прошлом семестре? Укажите число в часах'] = data['Сколько в среднем играли в прошлом семестре? Укажите число в часах'].apply(extract_hours)

X = data[signs]
y = data['Итоговая оценка за экзамен по ТОЭ']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42)


In [30]:
model = LinearRegression()

In [35]:
#обучаем модельку
X_train = X_train[y_train.notna()]
y_train= y_train.dropna()
imp = SimpleImputer(strategy='mean')
X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)
model.fit(X_train,y_train)

LinearRegression()

In [36]:
y_predict = model.predict(X_test) #предсказание на тестовой выборке

In [38]:
mean_abs_er = mean_absolute_error(y_test, y_predict)
r2 = r2_score(y_test, y_predict)

In [39]:
print("MAE = ", mean_abs_er)
print("R2 = ", r2)
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))

MAE =  0.6986032100871631
R2 =  -0.9442180837816003
